In [1]:
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import microatoll_sim.simulator as sim
import microatoll_sim.analytical_simulator as asim

## Setup


In [3]:
params = sim.SimParams(
    vert_shift=0.4,
    gr_mean=15,
    d=1.0,
    dt=0.1,  # will have 1/dt = 10 bands/yr
    T0=1980,
    delta_T=30,
    init_radius=0.3,
)


sl_df = pd.read_csv("./data/SeaLevel.csv", header=None, names=["time", "sl"])
sl_arr = sl_df.to_numpy()
sl_arr[:, 1] += params.vert_shift
band_sl_arr = sim.lowest_discreet(
    sl_arr, params.dt, params.T0, params.T0 + params.delta_T
)
gr_vec = np.ones(params.NT) * params.gr_mean

## Iterative simulator benchmarks


In [4]:
%timeit sim.coral_growth_all(params.init_radius, params.num_initial_pts, params.d, gr_vec, params.NT, band_sl_arr[:, 1])

174 ms ± 771 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%timeit sim.lowest_discreet(sl_arr, params.dt, params.T0, params.T0+params.delta_T)

75 µs ± 626 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [6]:
lines, living_statuses = sim.coral_growth_all(
    params.init_radius,
    params.num_initial_pts,
    params.d,
    gr_vec,
    params.NT,
    band_sl_arr[:, 1],
)
line = lines[-1]
living_status = living_statuses[-1]

In [8]:
%timeit sim.resample(line, living_status,params.d)

33.8 µs ± 3.86 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
%timeit sim.kill_loop(line, living_status,params.gr_mean) 

2.51 ms ± 33.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit sim.get_pointwise_unit_normals(line)

3.79 µs ± 6.85 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Analytical simulator benchmarks


In [7]:
gr = 15 * 1e-3
vert_shift = params.vert_shift
T0 = params.T0
delta_T = params.delta_T
init_radius = params.init_radius
dt = params.dt
sl_df = pd.read_csv("./data/SeaLevel.csv", header=None, names=["time", "sl"])
sl_arr = sl_df.to_numpy()
sl_arr[:, 1] += vert_shift
band_sl_arr = sim.lowest_discreet(sl_arr, 0.1, T0, T0 + delta_T)
x, y = band_sl_arr[:, 0], band_sl_arr[:, 1]


hlg_curve = [init_radius]
diedown_events = []
step = x[1] - x[0]
for i in range(len(x)):
    a, b = hlg_curve[-1] + gr * step, y[i]
    if a <= b:
        hlg_curve.append(a)
    else:
        hlg_curve.append(b)
        diedown_events.append((x[i], b))

diedown_events = np.array(diedown_events)
diedown_events = np.vstack([diedown_events, [x[-1], hlg_curve[-1]]])
init_time = diedown_events[0, 1] / gr
diedown_events[:, 0] = diedown_events[:, 0] - diedown_events[0, 0] + init_time
SLin = diedown_events
Nc = SLin.shape[0]  # Number of diedown events

# Arrays for arc segments
# First column:0:BF, 1:MF, 2:TF, 3:TB
# Third column: 0:Ox, 1:Oy, 2:radius, 3:1st endpoint, 4:2nd endpoint
Arcs = np.zeros((4, Nc, 5))

# Diedown point
# Second column: 0:Dx, 1:Dy, 2:Angle of normal vector,
#               3:Location of diedown (0:BF, 1:MF, 2:TF)
DD = np.zeros((Nc, 4))

# Surface trace
Sfc = np.zeros((100, 5))
# Initial shape and diedown
R = gr * SLin[0, 0]  # radius
Arcs[:, 0, 2] = R
Arcs[2, 0, 4] = np.pi / 2
Arcs[3, 0, 3] = np.pi / 2
Arcs[3, 0, 4] = np.pi / 2

DD[0, 0] = np.sqrt((R * R) - SLin[0, 1] * SLin[0, 1])
DD[0, 1] = SLin[0, 1]
DD[0, 2] = np.arctan2(DD[0, 1], DD[0, 0])
DD[0, 3] = 0

Sfc[0, :5] = [0, 0, R, DD[0, 2], np.pi / 2]
Isf = 0

In [8]:
%timeit asim.growth(Nc + 1, gr, Isf, SLin, Arcs, DD, Sfc)

7.06 µs ± 30.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
